# Collision detection and distances between volumes
The goal of this notebook is to introduce the **collision detection** and distance computation. It serves as a preamble to the notebook about rigid contact simulation, as collision detection is an essential part of simulating contacts.

In [1]:
import gepetuto.magic

## Set up

We will use several models in this tutorial:
- a simple scene with 3 convex objects (buildSceneThreeBodies)
- a variation of this first scene with more objects and walls (buildScenePillsBox)
- a stack of cubes of various size (buildSceneCubes)
- a robot hand

In [ ]:
from tp5.scenes import buildSceneThreeBodies, buildScenePillsBox, buildSceneCubes, buildSceneRobotHand

We rely on the HPP-FCL module of Pinocchio, which compute collision between geometries.

In [ ]:
import hppfcl
import pinocchio as pin
import numpy as np
import time
from schaeffler2025.meshcat_viewer_wrapper import MeshcatVisualizer
import matplotlib.pylab as plt

## A basic example of collisions

The goad of this section is to introduce a simple example of collision distances between bodies, and the underlying notions of witness points and segment and normal direction.

Let's build a simple scene with 3 objects and display their proximity.

In [ ]:
# %load tp5/generated/example_display_witness_build
# Build a scene
model,geom_model = buildSceneThreeBodies()
data = model.createData()
geom_data = geom_model.createData()

# Start meshcat
viz = MeshcatVisualizer(model=model, collision_model=geom_model,
                        visual_model=geom_model)


In [ ]:
q=pin.randomConfiguration(model)
viz.display(q)

In [ ]:
viz.viewer.jupyter_cell()

We can compute the distances between the 3 objects. Let's do it and show the closest points.

In [ ]:
from tp5.display_witness import DisplayCollisionWitnessesInMeshcat

In [ ]:
# %load tp5/generated/example_display_witness_witness
# Build the viewer add-on to display the witnesses.
mcWitnesses = DisplayCollisionWitnessesInMeshcat(viz)


In [ ]:
pin.computeDistances(model,data,geom_model,geom_data,q)
mcWitnesses.displayDistances(geom_data)

Each collision pair corresponds to a pair of closets points, respectively located at the surface of the collision geometries if they are not colliding. These points are sometime called the *witness* points.

The witness points are connected by the *witness* segment. This segment is normal to the two collision surfaces, in the case each surface is smooth around the witness point. The normalized direction is called the collision *normal*. Its orientation is a convention (the most logical convention is to go from body 1 to body 2 of the collision pair).

Let's move the objects to better see these witness elements.

In [ ]:
# %load tp5/generated/example_display_witness_trajectory
v = (np.random.rand(model.nv)*2-1)*1e-3
r0 = [ np.linalg.norm(q[7*i:7*i+3]) for i in range(model.nq//7) ]
for t in range(100):

    # Update the robot position along an arbitrary trajectory
    q = pin.integrate(model,q,v*10)
    for i in range(model.nq//7):
        q[7*i:7*i+3] *= r0[i]/np.linalg.norm(q[7*i:7*i+3])
    viz.display(q)

    # Display the witness points
    pin.computeDistances(model,data,geom_model,geom_data,q)
    mcWitnesses.displayDistances(geom_data)

    time.sleep(.01)


## Details about Pinocchio Coal (formerly HPP-FCL)

HPP-FCL is the former name of the library, now called COAL since 2025. You can find documentation about both.

Let's see now how to extract the contact and distance information from Pinocchio and the module HPP-FCL, and how to store this in a proper data structure.

### Geometry model
The geometry model contains a set of object, each described by a name, a geometry primitive and a placement with respect to a parent joint.

In [ ]:
geom1 = geom_model.geometryObjects[0]
geom1.name, geom1.parentJoint, geom1.geometry, geom1.placement

In addition, we also store the pairs of geometry objects that should be considered when evaluating collisions and distances.

In [ ]:
list(geom_model.collisionPairs)

### HPP-FCL computeDistances and computeCollisions

The geometry algorithms are implemented in HPP-FCL under two different sets of functions, which respectively compute the distance between bodies, and the collision between bodies.
When computing the distance, a unique pair of witness points is produced, and the signed distance is also evaluated.
When computing the collision, an effort is made to compute all the contact points, and early stop can be activated to reduce the algorithm cost as soon as a collision is found. 
All in all, for this initiation, both can be considered quite similarly.

Both functions are parametrized by a *request* object, and write their output in a *result* object. *Request* and *result* objects are preallocated in the geometry data, one of each for each pair of collisions. If you activate or deactivate a collision pair, you have to regenerate these objects (and so if you add a new geometry object in the list). 

In [ ]:
len(geom_model.collisionPairs), len(geom_data.collisionRequests), len(geom_data.distanceResults)

The placement of the geometry objects with respect to world frame are stored in geom_data.oMg, and computed as
$$^oM_g(q) = ^oM_i(q) ^iM_g$$
with $^oM_g$ the placement of the object wrt world, $^oM_i$ the placement of the parent joint and $^iM_g$ the (fixed) placement of the object wrt the parent joint.

In [ ]:
len(geom_model.geometryObjects),len(geom_data.oMg)

This computation is triggered by *pin.updateGeometryPlacements*, after forward kinematics has been run or by forcing the refresh of the forward kinematics.

In [ ]:
pin.updateGeometryPlacements(model,data,geom_model,geom_data,q)

The computation of the distances and collisions is triggered by their respective function and by default forces the kinematic update.

In [ ]:
pin.computeCollisions(model,data,geom_model,geom_data,q)
pin.computeDistances(model,data,geom_model,geom_data,q)

These two functions actually iterate over each pair of collisions and run *pin.computeDistance* or *pin.computeCollision*, which in turn runs *hppfcl.distance* and *hppfcl.collide*. 
HPP-FCL directly works with geometry placements, ignoring the kinematics and the configuration space which is provided by Pinocchio.
All these functions have very similar signatures, and Pinocchio is mostly doing a basic wrapping around the HPP-FCL library and gently connects the forward kinematics with the collision algorithms. 

### The *request* objects
For collisions, Pinocchio mostly forward the requests to the library HPP-FCL, recently renamed COAL. It is possible to parametrize the algorithm computing the collision by setting the *request* objects stored in the geometry data, one for each collision pair:

In [ ]:
for req in geom_data.collisionRequests:
    req.security_margin = 1e-3 # Collision is detected with a millimeter margin
    req.num_max_contacts = 5 # Max number of collision point
    req.enable_contact = True # To force the computation of the normal


The same is true for the *distance* algorithm.

In [ ]:
for req in geom_data.distanceRequests:
    req.security_margin = 1e-3 # Collision is detected with a millimeter margin
    req.num_max_contacts = 5 # Max number of collision point
    req.enable_contact = True # To force the computation of the normal

Then, re-evaluate the collisions.

In [ ]:
pin.computeCollisions(model,data,geom_model,geom_data,q)
pin.computeDistances(model,data,geom_model,geom_data,q)

### The *result* objects
The distance result contains the pair of witness points $p_1$ and $p_2$, the normal direction pointing from $p_1$ to $p_2$ and the signed distance.

In [ ]:
d = geom_data.distanceResults[0]
print("p1:",d.getNearestPoint1())
print("p2:",d.getNearestPoint2())
print("n:",d.normal)
print("dist:",d.min_distance)
print("check:",np.cross(d.normal,d.getNearestPoint2()-d.getNearestPoint1())) # The two vectors are parallel.

The collision object contains similar information, but can stores several witness pairs instead of a single one, or none if there is no collisions.
The collision is decided based on a security margin, tuned in the collision request object.

In [ ]:
print('Margin:',geom_data.collisionRequests[0].security_margin)
c = geom_data.collisionResults[0]
print('Number of collisions:',len(c.getContacts()))

Now choose a configuration in collision and look at the content of this contact list.

In [ ]:
# %load tp5/generated/example_find_collisions_find
for trial in range(1000):
    q = pin.randomConfiguration(model)
    col = pin.computeCollisions(model, data, geom_model, geom_data, q)
    if col:
        break
assert col
viz.display(q)


In [ ]:
# %load tp5/generated/example_find_collisions_print
for pairId,c in enumerate(geom_data.collisionResults):
    if len(c.getContacts())>0:
        contact = c.getContact(0)
        print([ n for n in dir(contact) if '__' not in n])
        break


## From collision detection to contact model

### The maths
Once the collision are detected, we have to decide the model of the contact interaction that will happen at this location. It can range from a fixed 3D contact (bilateral), a sliding contact enforcing only constraints along the normal direction, a frictional contact or even a full 6D contact with no degree of freedom between the two corresponding bodies (and many other models you can imagine).
To describe the contact model, we best formulate it in a reference frame at the contact location. Here we arbitrarily decide to align the *z* direction of the contact frame with the normal direction (oriented from body 1 to body 2) and set the two other *x* and *y* directions to any arbitrary orthogonal basis of the tangential plane. If the collision distance is exactly 0 (which will never be), the center of the frame should be at the contact point. In practice, we can define either two contact frames at each witness point or a single one at the middle point between both. Since the contact distance will never be large, it should be approximately the same and both solutions are acceptable in practice.

In [ ]:
OC1 = contact.getNearestPoint1()
OC2 = contact.getNearestPoint2()
normal = contact.normal

From these vectors, build the two contact frames $^oM_{c1}$ and $^oM_{c2}$ centered in $c_1$  and $c_2$ and with *z* axis aligned with the *normal* direction. You can for example use the pin.Quaternion.FromTwoVectors(v1,v2) function, which returns a rotation $r(.)$ such that $r(v_1)=v_2$ ie that transforms $v_1$ into $v_2$ (here we want to transform the *z* axis $z=[0,0,1]$ into the *normal* vector).

In [ ]:
# %load tp5/generated/create_rigid_contact_models_for_hppfcl_frames
pair = geom_model.collisionPairs[pairId]
gid1, gid2 = pair.first, pair.second
g1 = geom_model.geometryObjects[gid1]
g2 = geom_model.geometryObjects[gid2]
jid1 = g1.parentJoint
jid2 = g2.parentJoint
oMj1 = data.oMi[jid1]
oMj2 = data.oMi[jid2]

# Compute translation and rotation of the contact placements
# If dist=0, both placements are identical (and should be somehow close
# when dist is reasonibly small).
quat = pin.Quaternion.FromTwoVectors(
    pin.ZAxis, normal
)  # orientation of the contact frame wrt world
assert np.isclose(quat.norm(), 1)
oMc1 = pin.SE3(quat.matrix(), OC1)  # Placement of first contact frame in world
oMc2 = pin.SE3(quat.matrix(), OC2)  # Placement of second contact frame in world


The pin.RigidConstaintModel class in Pinocchio conveniently stores these contact quantities: the two parent joints that carries the corresponding colliding bodies, the placements of the contact frames with respect to the joint frames $^{j1}M_{c1}$, $^{jc}M_{c2}$, and a flag specifying if the contact model is 3D (point contact) or 6D (surface contact), which will later be convenient to evaluate the corresponding Jacobian.

In [ ]:
# %load tp5/generated/create_rigid_contact_models_for_hppfcl_model
contact_model = pin.RigidConstraintModel(
        pin.ContactType.CONTACT_3D,
        model,
        jid1,oMj1.inverse()*oMc1,
        jid2,oMj2.inverse()*oMc2,
        pin.LOCAL)


In [ ]:
contact_data = contact_model.createData()

### Creation wrapper
This code has been conveniently wrapped in a dedicated function, that takes all the collision or distance results and returns a list of contact models.

In [ ]:
from tp5.create_rigid_contact_models_for_hppfcl import createContactModelsFromCollisions,createContactModelsFromDistances

In [ ]:
# %load tp5/generated/create_rigid_contact_models_for_hppfcl_example
pin.computeCollisions(model,data,geom_model,geom_data,q,False)
contact_models = createContactModelsFromCollisions(model,data,geom_model,geom_data)
contact_datas = [ cm.createData() for cm in contact_models ]

pin.computeDistances(model,data,geom_model,geom_data,q)
contact_models = createContactModelsFromDistances(model,data,geom_model,geom_data,
                                                       threshold=10) # threshold in meter
contact_datas = [ cm.createData() for cm in contact_models ]


### Display the contact models
A robot description (like the robot URDF) typically describes two sets of geometries: a first set, accurate, detailled and texture fore visualization and non real-time evaluations (called the *visual* model); a second one, less detailed, sometimes approximated or convexified, for efficient computations (often called the *collision* model).

Let's display in the 3D viewer the contact models with red contact patches (disks) located at the center of the contact frames and aligned with the *x-y* contact plane. 
To make it easy to render, we will put these patches in the *visual* model of the system. 
A fix set of patches is preallocated at initialization (and hidden or displaced far from the scene center when not useful). When contacts are created, the patches are revealed and properly place for rendering.
The functions in the file display_collision_patches have be written for that.

In [ ]:
from tp5.display_collision_patches import preallocateVisualObjects,updateVisualObjects

In [ ]:
# %load tp5/generated/example_display_collision_patches_create
# Obtained by simply copying the collision model
visual_model = geom_model.copy()
preallocateVisualObjects(visual_model)

# Start meshcat
viz = MeshcatVisualizer(model=model, collision_model=geom_model,
                        visual_model=visual_model)


In [ ]:
viz.viewer.jupyter_cell()

In [ ]:
# %load tp5/generated/example_display_collision_patches_display
updateVisualObjects(model,data,contact_models,contact_datas,visual_model,viz)
viz.display(q)


You have to run updateVisualObjects each time you change q, and before calling viz.display(q).

##  Building a (very) simple kinematic simulator

Let's build a very simple collision simulator. Starting from an initial configuration where some bodies are colliding, you have to choose a velocity of each body to disentangle the colliding parts.
The scene created above is composed of 3 bodies, each attached to a "free-flyer" joint. The configuration of the scene is the concatenation of the configuration of each joint, corresponding to translation and quaternion describing the placement of each body. The configuration velocity also is the concatenation of the velocity of each body, each corresponding to the spatial velocity $\nu = (v,\omega)$ of each body expressed in the local frame. 

In [ ]:
# %load tp4/generated/example_separate_objects_hyperparams
# HYPER PARAMETERS OF THE PUSH STRATEGY
PUSH_FACTOR = .1
EPSILON = 1e-1
NB_ITER = 100
# Compute the contact information based on distances or collisions?
USE_DISTANCE = True


At each iteration, you have to choose a configuration velocity, then integrate it.

In [ ]:
# Set minimal distance to be EPSILON
# (can be zero, but the rendering is less clear).
for r in geom_data.collisionRequests:
    r.security_margin = EPSILON

# Keep distance history for active pairs (indexed by contact name)
h_dist = {}

# Iteratively push the colliding pairs ...
for i in range(NB_ITER):

    # We will compute a change of configuration dq.
    # 0 if no active pair.
    dq = np.zeros(model.nv)

    # Compute the collision at current configuration.
    if USE_DISTANCE:
        pin.computeDistances(model,data,geom_model,geom_data,q)
    else:
        pin.computeCollisions(model,data,geom_model,geom_data,q)

    # From hppfcl contact information, build a pin.RigidContactModel
    if USE_DISTANCE:
        contact_models = createContactModelsFromDistances(model,data,geom_model,geom_data,EPSILON)
    else:
        contact_models = createContactModelsFromCollisions(model,data,geom_model,geom_data)
    contact_datas = [ cm.createData() for cm in contact_models ]

    # For each detected contact ...
    for cmodel,cdata in zip(contact_models,contact_datas):

        # Recover contact information
        jid1 = cmodel.joint1_id
        j1Mc1 = cmodel.joint1_placement
        jid2 = cmodel.joint2_id
        j2Mc2 = cmodel.joint2_placement

        # Compute signed distance
        oMc1 = cdata.oMc1 = data.oMi[jid1]*j1Mc1
        oMc2 = cdata.oMc2 = data.oMi[jid2]*j2Mc2
        dist = oMc1.actInv(oMc2.translation)[2]-EPSILON  # signed distance
        
        # ### TODO
        # Here, do something to the velocity of the two joints carrying the 
        # two colliding bodies.
        # Displacement for body 1
        dq[model.idx_vs[jid1]:model.idx_vs[jid1]+6] += 0 ### TODO FIX ME
        # Displacement for body 2
        dq[model.idx_vs[jid2]:model.idx_vs[jid2]+6] -= 0 ### TODO FIX ME
        
        # Log the distance in h_dist for future plot
        if cmodel.name not in h_dist:
            h_dist[cmodel.name] = np.zeros(NB_ITER)
        h_dist[cmodel.name][i] = dist

    # Finally, modify the current config q with the push dq
    q = pin.integrate(model,q,dq)

    # Display the current configuration
    if i % 10 == 0:
        # Meshcat is slow to display the patches, display once in a while
        updateVisualObjects(model,data,contact_models,contact_datas,visual_model,viz)
        viz.display(q)


If you don't have any strong ideas, you can start to compute a "push" repulsive 3D velocity that the first witness point should adopt. The second witness point should likely adopt the opposite velocity. This is the value of the velocity vector field at the contact point. Now choose the spatial velocity at the center of the body so that the value of the corresponding vector field matches. In a first trial, a spatial velocity with 0 angular velocity $\omega$ should be evident to guess. Can you imagine a more efficient linear+angular velocity that would work as well?

In [ ]:
%do_not_load_snippet tp5/generated/example_separate_objects_loop

### With a more complex scene
Now let's load a square jar full of pills. Run the same algorithm on it. 

In [ ]:
# %load tp5/generated/scenes_pills
model,geom_model = buildScenePillsBox(seed=2,nobj=30,wall_size=2.0,one_of_each=True)
visual_model = geom_model.copy()
viz = MeshcatVisualizer(model=model, collision_model=geom_model,
                        visual_model=geom_model)

# Generate colliding configuration
data = model.createData()
geom_data = geom_model.createData()
for i in range(10):
    q0 = pin.randomConfiguration(model)
    pin.computeCollisions(model,data,geom_model,geom_data,q0)
    if sum([ len(c.getContacts()) for c in geom_data.collisionResults ])>10:
        break
    print(sum([ len(c.getContacts()) for c in geom_data.collisionResults ]))


In [ ]:
q = q0.copy()
viz.display(q)
viz.viewer.jupyter_cell()

You can plot the convergence of the colliding bodies from initial negative (penetration) distance to 0.

In [ ]:
# %load tp4/generated/example_separate_objects_plot
# Plot the distances
for k,v in h_dist.items():
    h = plt.plot(v,label=k)
plt.legend()
